<a href="https://colab.research.google.com/github/quickstunt/data_proc/blob/main/twitter_airticles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#当日公開分のみ用
import pandas as pd

#データの読み込み
#PVのCSVファイル
input_csv_file = "/content/20220816-20220816.csv"
#公開記事が一覧してあるCSVファイル
input_article_file = "/content/8_16.csv"
#投稿した記事ID
targetID_path = "https://URL/article/ArticleID/"

#データの変換
data_original = pd.read_csv(input_csv_file)
data_articles = pd.read_csv(input_article_file)
test_date = input_csv_file[9:26]
targetID = targetID_path[32:40]

#「時間帯」「記事ID」「ページビュー数」の取り出し
data_master = data_original[["時間帯", "記事ID", "ページビュー数"]]

#「時間帯」を文字列に変換
data_master["時間帯"] = data_master["時間帯"].astype(str)

#各項目で欠損値の有無を確認
data_master.isnull().any(axis = 0)

#data_masterを読み込んで、記事IDの形式を揃え、投稿記事IDを取り除く
data_master["記事ID"] = data_master["記事ID"].str[:18]
data_targetID_locs = data_master["記事ID"].str.contains(targetID)
data_nontargetID = data_master.loc[~data_targetID_locs]

#当日記事IDの形式を整える
data_articles["記事ID"] = data_articles["記事ID"].str.replace("https://URL/article/", "")
data_articles["記事ID"] = data_articles["記事ID"].str.replace("/", "")

#当日記事IDから投稿記事IDを取り除く
data_targetID_articles_locs = data_articles["記事ID"].str.contains(targetID)
data_articles["記事ID"] = data_articles.loc[~data_targetID_articles_locs]
data_articles.dropna(inplace=True)

#当日記事IDから投稿記事IDを取り除いたものをarticleIDsとする
articleIDs = data_articles["記事ID"]

#当日記事IDのデータを保存するデータフレームの定義
data_articleIDs_temp = pd.DataFrame(index = ["時間帯", "記事ID", "ページビュー数"], columns = [])
data_articleIDs = pd.DataFrame(index = ["時間帯", "記事ID", "ページビュー数"], columns = [])

#当日記事IDのデータをデータフレームに格納
for articleID in articleIDs:
  data_articleID_locs = data_nontargetID["記事ID"].str.contains(articleID)
  data_articleIDs_temp = data_nontargetID.loc[data_articleID_locs]
  data_articleIDs = pd.concat([data_articleIDs, data_articleIDs_temp], ignore_index = True)

data_articleIDs.dropna(inplace=True)

#当日記事IDの投稿数を計算
pages_per_time2 = data_articleIDs.groupby("時間帯").nunique()["記事ID"]
pv_per_time2 = data_articleIDs.groupby("時間帯").sum()["ページビュー数"]
pv_mean_by_time2 = pv_per_time2 / pages_per_time2

#pv_mean_by_time2をデータフレーム化
data_articleIDs_output = pd.DataFrame(pv_mean_by_time2)

#data_output.columnsの列名として「投稿以外平均ページビュー」を付与
data_articleIDs_output.columns = ['投稿以外平均ページビュー']


#読まれてます投稿分のページビュー

#data_masterを読み込んで、記事IDの形式を揃え、投稿記事IDのみを取り出す
data_master["記事ID"] = data_master["記事ID"].str[:18]
data_targetID_locs = data_master["記事ID"].str.contains(targetID)
data_targetID_master = data_master.loc[data_targetID_locs]

#出力項目を「時間帯」「ページビュー数」にした出力用データフレームを定義
data_targetID_output = data_targetID_master[["時間帯", "ページビュー数"]]

#data_articleIDs_outputとdata_targetID_outputをマージ
data_output = pd.merge(data_articleIDs_output, data_targetID_output, on = "時間帯", how='outer')

#結合時に時間帯の並びが変わる可能性があるので、時間帯でソート
data_output = data_output.sort_values(by = ["時間帯"])

#データフレームにビューがない時間帯を付け加える
import numpy as np

#年月日をファイル名から抽出
test_date_single = input_csv_file[9:17]

#24時間帯の仮データ（0）を持つダミーのデータフレームを作成
data_dummy_24 = pd.DataFrame(np.array([[test_date_single + "00",0],
                         [test_date_single + "01",0],
                         [test_date_single + "02",0],
                         [test_date_single + "03",0],
                         [test_date_single + "04",0],
                         [test_date_single + "05",0],
                         [test_date_single + "06",0],
                         [test_date_single + "07",0],
                         [test_date_single + "08",0],
                         [test_date_single + "09",0],
                         [test_date_single + "10",0],
                         [test_date_single + "11",0],
                         [test_date_single + "12",0],
                         [test_date_single + "13",0],
                         [test_date_single + "14",0],
                         [test_date_single + "15",0],
                         [test_date_single + "16",0],
                         [test_date_single + "17",0],
                         [test_date_single + "18",0],
                         [test_date_single + "19",0],
                         [test_date_single + "20",0],
                         [test_date_single + "21",0],
                         [test_date_single + "22",0],
                         [test_date_single + "23",0],
                         ]),
                        columns=['時間帯', 'ページビュー数ダミー'])

#仮のデータフレームと出力結果をマージ
data_merged_24 = pd.merge(data_dummy_24, data_output, on = "時間帯", how = "outer")
#「時間帯」「投稿以外平均ページビュー」「ページビュー数」だけを出力結果とし、欠損値は0とする
data_out_24 = data_merged_24.fillna("0")[["時間帯", "投稿以外平均ページビュー", "ページビュー数"]]

#欠損値を0にしてCSVファイルに出力
output_csv_file5 = "output_" + test_date + "_5.csv"
data_out_24.fillna("0").to_csv(output_csv_file5)